In [ ]:
import geopandas as gpd
import os
import pandas as pd
from shapely.geometry import Point
import zipfile

#### Add the folder path where **Gedi L4A** csv's were downloaded in `path_` variable
#### These csv's will be merged and then converted to one single .shp file

In [ ]:
path_ = "/EarthLab/BulkDownload/agbd_output/2021/"
sorted_ = sorted(os.listdir(path_), key=lambda x: int(x.split(".csv")[0]))

#### Add where to save converted csv file to .shp file in `merged_shapefile_path` variable
#### Optionally add the shape file name in `merged_shapefile_name` variable

In [ ]:
# Read every csv file
# Convert it into a Geo Dataframe and add it to a running "merged" geo-dataframe
# Save the Geo Dataframe as a .shp file


merged_shapefile_path = 'SHP_DATA_FOLDER_L4A_2021'
merged_shapefile_name = 'merged_shapefile.shp'

def read_csv_and_merge_shp():
  gdfs = []
  cols= ['lat_lowestmode',
 'lon_lowestmode',
 'elev_lowestmode',
 'shot_number',
 'agbd',
 'l4_quality_flag',
 'degrade_flag',
 'agbd_se',
 'land_cover_data/pft_class',
 'solar_elevation']
  for index, file_ in enumerate(sorted_):
    print(file_)
    csv_file_path = os.path.join(path_, file_)
    df = pd.read_csv(csv_file_path)
    computed_columns =  df.columns.tolist()
    if computed_columns != cols:
      df = pd.read_csv(csv_file_path, names=cols)
    df['geometry'] = df.apply(lambda row: Point(row['lon_lowestmode'], row['lat_lowestmode']), axis=1)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.set_crs(epsg=4326, inplace=True)
    gdfs.append(gdf)
  print("done creating all gdfs")
  base_crs = gdfs[0].crs
  for i in range(1, len(gdfs)):
      if gdfs[i].crs != base_crs:
          gdfs[i] = gdfs[i].to_crs(base_crs)
  merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
  os.makedirs(merged_shapefile_path, exist_ok=True)
  merged_gdf.to_file(os.path.join(merged_shapefile_path, merged_shapefile_name))
  print(f"Merged shapefile created at {merged_shapefile_path}")

In [ ]:
read_csv_and_merge_shp()